## Importing libraries

In [ ]:
import numpy as np
import pandas as pd

## Making new directory

In [ ]:
!mkdir mnist_test
!mkdir mnist_train

## Unziping the data

In [ ]:
!unzip test.zip -d mnist_test

In [ ]:
!unzip train.zip -d mnist_train

## Images in the train and test set 

In [ ]:
import os

print(len(os.listdir('/content/mnist_train/train')))
print(len(os.listdir('/content/mnist_test/test')))

49000
21000


In [ ]:
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/Test.csv')


In [ ]:
train.head()

,filename,label
0,0.png,4
1,1.png,9
2,2.png,1
3,3.png,7
4,4.png,3


In [ ]:
train.label.unique()

array([4, 9, 1, 7, 3, 2, 6, 0, 8, 5])

In [ ]:
train.label=train.label.astype(str)

In [ ]:
test.head()

,filename
0,49000.png
1,49001.png
2,49002.png
3,49003.png
4,49004.png


## Using data flow from dataframe and model builidng

In [ ]:
# flow from dataframe

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten,Conv2D,MaxPooling2D
from keras.models import Sequential


train_datagen=ImageDataGenerator(rescale=1.0/255.0,validation_split=0.20)
test_datagen=ImageDataGenerator(rescale=1.0/255.0)

train_generator= train_datagen.flow_from_dataframe(dataframe=train,directory='/content/mnist_train/train',
                                                   x_col='filename',
                                                   y_col='label',
                                                   seed=7,
                                                   shuffle=True,
                                                   subset='training',
                                                   class_mode='categorical',
                                                   target_size=(28,28),batch_size=32)

test_generator= test_datagen.flow_from_dataframe(dataframe=test,directory='/content/mnist_test/test',
                                                   x_col='filename',
                                                   y_col='label',
                                                   seed=7,
                                                   shuffle=False,
                                                   class_mode=None,
                                                   target_size=(28,28),batch_size=32)

val_generator= train_datagen.flow_from_dataframe(dataframe=train,directory='/content/mnist_train/train',
                                                   x_col='filename',
                                                   y_col='label',
                                                   seed=7,
                                                   shuffle=True,
                                                   subset='validation',
                                                   class_mode='categorical',
                                                   target_size=(28,28),batch_size=32)


model=Sequential([Conv2D(32,(3,3),activation='relu',input_shape=(28,28,3),padding='same'),
                  MaxPooling2D(),
                  Flatten(),
                  Dense(1000,activation='relu'),
                  Dense(512,activation='relu'),
                  Dense(10,activation='softmax')])
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()


Found 39200 validated image filenames belonging to 10 classes.
Found 21000 validated image filenames.
Found 9800 validated image filenames belonging to 10 classes.
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 1000)              6273000   
_________________________________________________________________
dense_7 (Dense)              (None, 512)               512512    
_________________________________________________________________
dense_8 (Dense)       

## Training the model

In [ ]:
history=model.fit(train_generator,epochs=10,verbose=1,validation_data=val_generator)

Epoch 1/10
1225/1225 [==============================] - 100s 82ms/step - loss: 0.4292 - accuracy: 0.8834 - val_loss: 0.2019 - val_accuracy: 0.9400
Epoch 2/10
1225/1225 [==============================] - 99s 81ms/step - loss: 0.1678 - accuracy: 0.9511 - val_loss: 0.1520 - val_accuracy: 0.9543
Epoch 3/10
1225/1225 [==============================] - 100s 82ms/step - loss: 0.1227 - accuracy: 0.9624 - val_loss: 0.1210 - val_accuracy: 0.9628
Epoch 4/10
1225/1225 [==============================] - 100s 81ms/step - loss: 0.0952 - accuracy: 0.9720 - val_loss: 0.1043 - val_accuracy: 0.9669
Epoch 5/10
1225/1225 [==============================] - 100s 82ms/step - loss: 0.0781 - accuracy: 0.9760 - val_loss: 0.0929 - val_accuracy: 0.9711
Epoch 6/10
1225/1225 [==============================] - 100s 82ms/step - loss: 0.0653 - accuracy: 0.9806 - val_loss: 0.0816 - val_accuracy: 0.9747
Epoch 7/10
1225/1225 [==============================] - 99s 81ms/step - loss: 0.0553 - accuracy: 0.9828 - val_loss: 0.0

## Evaluation on validation data

In [ ]:
model.evaluate_generator(val_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.065483458340168, 0.9776530861854553]

## Predictions on test images

In [ ]:
prediction=model.predict_generator(test_generator)
prediction

Instructions for updating:
Please use Model.predict, which supports generators.


array([[5.97509056e-11, 2.06240892e-11, 5.05024356e-10, ...,
        5.13266855e-07, 5.45950063e-10, 2.38546272e-09],
       [9.51697707e-01, 1.22449865e-05, 3.93448994e-02, ...,
        7.43489290e-05, 4.69653588e-03, 3.35158128e-03],
       [4.70837898e-04, 8.33208219e-07, 4.63897959e-05, ...,
        3.51912603e-02, 1.50952517e-04, 8.64626586e-01],
       ...,
       [6.57906241e-07, 1.32646633e-07, 3.39706094e-06, ...,
        1.45432919e-08, 3.14862591e-05, 2.75041367e-09],
       [4.13936718e-10, 1.95406269e-08, 4.41152270e-06, ...,
        2.63561475e-08, 8.36389802e-08, 9.78840620e-12],
       [3.11520921e-06, 5.04484376e-07, 9.99824703e-01, ...,
        2.25828938e-08, 2.46526022e-09, 8.45165804e-09]], dtype=float32)

In [ ]:
prediction.shape

(21000, 10)

In [ ]:
pred=[]
for i in range(0,21000):
  pred.append(np.argmax(prediction[i]))
  

In [ ]:
submission=pd.DataFrame()
submission['filename']=test['filename']
submission['label']=pred


In [ ]:
submission.head()

,filename,label
0,49000.png,4
1,49001.png,0
2,49002.png,9
3,49003.png,7
4,49004.png,9


In [ ]:
submission.to_csv('/content/submission.csv')